In [1]:
from pprint import pprint
import pandas as pd

In [2]:
from requests_futures.sessions import FuturesSession

In [3]:
from tracking_grants import articles_f, email, ncbi_api_key, tool_name

In [4]:
from ratelimit import limits, sleep_and_retry
from tqdm.auto import tqdm
import re

In [5]:
articles = pd.read_csv(articles_f)

In [6]:
dois = articles.DOI.sample(100).unique().tolist()

### Async calls with request futures

In [79]:
NCBI_CALLS_PER_SEC = 3

class Eutils:
    def __init__(self, tool, email, api_key):
        self.search_api = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        self.session = FuturesSession()
        self.session.hooks['response'] = self.response_hook
        
        self.params = {
            "tool": tool,
            "email": email,
            "api_key": api_key,
            "db": "pubmed",
            "retmax": 1,
        }

    @sleep_and_retry
    @limits(calls=NCBI_CALLS_PER_SEC, period=1)
    def search(self, doi):
        params = self.params
        params["term"] = doi

        future = self.session.get(self.search_api, params=params)
        return future
    
    @staticmethod
    def response_hook(resp, *args, **kwargs):
        if resp.status_code != 200:
            resp.data = f"Error:{response.status_code}"
        else:
            text = resp.text
            resp.data = None

            # Only return DOI has been found in the text
            if 'PhraseNotFound' not in text:
                count = int(re.search(r"<Count>(\d+)<\/Count>", text).group(1))
                # Only return unique matches
                if count == 1:
                    match = re.search(r"<Id>(\d+)<\/Id>", text)
                    if match:
                        resp.data =  match.group(1)

In [80]:
eutils = Eutils(tool_name, email, ncbi_api_key)

In [81]:
futures = []
for doi in tqdm(dois):
    futures.append((doi, eutils.search(doi)))

In [82]:
results = {}
for doi, future in tqdm(futures):
    response = future.result()
    results[doi] = response.data

### Async Code with aiohttp

In [93]:
import aiohttp
import asyncio
from asyncio_throttle import Throttler

In [94]:
import time

In [101]:
COUNT_REGEX = r"<Count>(\d+)<\/Count>"
ID_REGEX = r"<Id>(\d+)<\/Id>"

class Eutils():
    def __init__(self, tool, email, api_key, loop, calls_per_sec=3):
        self.baseurl = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        self.params = {
            "tool": tool,
            "email": email,
            "api_key": api_key,
            "db": "pubmed",
            "retmax": 1,
        }
        
        self.loop = loop
        self.throttler = Throttler(rate_limit=calls_per_sec, period=1)
    
    async def __fetch(self, session, params):
        async with self.throttler:
            async with session.get(self.baseurl, params=params) as resp:
                await asyncio.sleep(1)
                return await resp.text()
#         await asyncio.sleep(0.05)
                
    async def __parse(self, text):
        pmid = None
        if 'PhraseNotFound' not in text:
            count = int(re.search(COUNT_REGEX, text).group(1))
            # Only return unique matches
            if count == 1:
                match = re.search(ID_REGEX, text)
                if match:
                    pmid = match.group(1)
        return pmid

    async def get_pmid(self, session, doi):
        params = self.params
        params['term'] = doi
        
        text = await self.__fetch(session, params)
        pmid = await self.__parse(text)
        return (doi, pmid)

    async def run(self, dois):
        tasks = []
        async with aiohttp.ClientSession(loop=self.loop) as session:
            # Create tasks with DOI
            tasks = [asyncio.ensure_future(self.get_pmid(session, doi)) for doi in dois]  

            # Await tasks and print progress
            responses = [await t for t in tqdm(asyncio.as_completed(tasks), total=len(dois))]

            return responses

In [102]:
loop = asyncio.get_event_loop()
eutils = Eutils(tool_name, email, ncbi_api_key, loop)

In [103]:
dois = articles.DOI.sample(100).unique().tolist()

In [104]:
await eutils.run(dois)

[('10.1593/neo.06331', '16984734'),
 ('10.1158/0008-5472.can-05-2630', '16322249'),
 ('10.1016/j.cell.2006.03.027', '16603237'),
 ('10.4049/jimmunol.165.4.1956', '17186423'),
 ('10.1155/2012/185435', '17186423'),
 ('10.1210/endo.142.5.8141', '17186423'),
 ('10.1016/j.ccr.2006.10.003', '17186423'),
 ('10.2741/1986', '17186423'),
 ('10.1586/eem.11.24', '17186423'),
 ('10.1002/(sici)1096-8628(20000529)92:3<224::aid-ajmg12>3.0.co;2-j',
  '17186423'),
 ('10.1021/pr800468j', '17186423'),
 ('10.1038/modpathol.2016.88', '17186423'),
 ('10.1016/s0962-8924(01)02179-1', '17186423'),
 ('10.1038/sj.onc.1208110', '17186423'),
 ('10.1021/bm101036f', '17186423'),
 ('10.1128/mcb.20.23.8969-8982.2000', '17186423'),
 ('10.1016/j.biomaterials.2012.09.048', '17186423'),
 ('10.1126/scisignal.2000029', '17186423'),
 ('10.1002/jcb.25768', '17186423'),
 ('10.1016/j.jmatprotec.2013.05.011', '17186423'),
 ('10.1172/jci200421107', '17186423'),
 ('10.1530/erc-12-0085', '17186423'),
 ('10.3389/fimmu.2017.00563', '1